In [2]:
from importlib.metadata import distribution
import sys
from collections import defaultdict
import random

validCharacters = set("1234567890abcdefghijklmnopqrstuvwxyz. ")
def preprocess_line(line):
    valid = "##"
    #.lower() automatically changes all capitals to lower.
    for i in line.lower():
        if i in validCharacters:
            if i in set("1234567890"):
                valid += "0"
            else:
                valid += i
    valid += "#"
    return valid

In [ ]:
#Add Alpha ESTIMATION as the Easiest
#Need to deal with newLine and endLine
def modelling(infile, alpha):
    tri_counts= defaultdict(int)
    bi_counts = defaultdict(int)
    model = dict()
    with open(infile) as f:
        for line in f:
            line_ = preprocess_line(line)
            
            for j in range(len(line_)-2):
                trigram = line_[j:j+3]
                tri_counts[trigram] += 1
                
                #Get Bigrams to Find Prob
                bigram = line_[j:j+2]
                bi_counts[bigram] += 1
            
            f = len(line_) - 3
            bigram = line_[f:f+2]
            bi_counts[bigram] += 1

    #If we consider a|bc, bi_counts contains all trigram counts of bc occuring.
    #Switch to add alpha of 0.05
    for state in bi_counts:
        #build probability distribution
        distribution = []
        letter = []
        for c in validCharacters:
            #state+c is the trigram
            prob = (tri_counts.get(str(state+c),0) + alpha) / (bi_counts[state] + (len(validCharacters)*alpha))
            distribution.append(prob)
            letter.append(c)
        #Extra Operation for # eol
        prob = (tri_counts.get(str(state+'#'),0) + alpha) / (bi_counts[state] + (len(validCharacters)*alpha))
        distribution.append(prob)
        letter.append("#")
            
        #We seperate the two rather than using a tuple so we can easily weigh later on.
        model[state] = (letter, distribution)

    return model

In [ ]:
def generate_from_LM(model):
    fullState = ["#","#"]
    for j in range(1,300):
        state = fullState[j] + fullState[j-1]
        nextState = random.choices(model[state][0], model[state][1])
        fullState.append(nextState)
    print(str(fullState))
    return
 
def printModel(model):
    for state in sorted(model.keys()):
        for i in range(len(validCharacters)):
            print(state + model[state][0][i] + "|" + str(model[state][1][i]))
        
        #print("(" + str(trigram[2]) + "|" + str(trigram[:2]) + "):" + str(model[trigram]))

In [ ]:
model = modelling('training.en', 0.1)
printModel(model)
#generate_from_LM(model)